# 第8章 分类数据

##### 前期预备

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('joyful-pandas-master/data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、category的创建及其性质
### 1. 分类变量的创建
#### （a）用Series创建

In [2]:
pd.Series(["a", "b", "c", "a"], dtype="category")

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

#### （b）对DataFrame指定类型创建

In [3]:
temp_df = pd.DataFrame({'A':pd.Series(["a", "b", "c", "a"], dtype="category"),'B':list('abcd')})
temp_df.dtypes

A    category
B      object
dtype: object

#### （c）利用内置Categorical类型创建

In [4]:
cat = pd.Categorical(["a", "b", "c", "a"], categories=['a','b','c'])
pd.Series(cat)

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

#### （d）利用cut函数创建
#### 默认使用区间类型为标签

In [5]:
pd.cut(np.random.randint(0,60,5), [0,10,30,60])

[(30, 60], (10, 30], (30, 60], (10, 30], (10, 30]]
Categories (3, interval[int64]): [(0, 10] < (10, 30] < (30, 60]]

#### 可指定字符为标签

In [6]:
pd.cut(np.random.randint(0,60,5), [0,10,30,60], right=False, labels=['0-10','10-30','30-60'])

[0-10, 30-60, 10-30, 0-10, 30-60]
Categories (3, object): [0-10 < 10-30 < 30-60]

### 2. 分类变量的结构
#### 一个分类变量包括三个部分，元素值（values）、分类类别（categories）、是否有序（order）
#### 从上面可以看出，使用cut函数创建的分类变量默认为有序分类变量
#### 下面介绍如何获取或修改这些属性
#### （a）describe方法
#### 该方法描述了一个分类序列的情况，包括非缺失值个数、元素值类别数（不是分类类别数）、最多次出现的元素及其频数

In [7]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.describe()

count     4
unique    3
top       a
freq      2
dtype: object

#### （b）categories和ordered属性
#### 查看分类类别和是否排序

In [8]:
s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [9]:
s.cat.ordered

False

### 3. 类别的修改

#### （a）利用set_categories修改
#### 修改分类，但本身值不会变化

In [10]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.set_categories(['new_a','c'])

0    NaN
1    NaN
2      c
3    NaN
4    NaN
dtype: category
Categories (2, object): [new_a, c]

#### （b）利用rename_categories修改
#### 需要注意的是该方法会把值和分类同时修改

In [11]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.rename_categories(['new_%s'%i for i in s.cat.categories])

0    new_a
1    new_b
2    new_c
3    new_a
4      NaN
dtype: category
Categories (4, object): [new_a, new_b, new_c, new_d]

#### 利用字典修改值

In [12]:
s.cat.rename_categories({'a':'new_a','b':'new_b'})

0    new_a
1    new_b
2        c
3    new_a
4      NaN
dtype: category
Categories (4, object): [new_a, new_b, c, d]

#### （c）利用add_categories添加

In [13]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.add_categories(['e'])

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (5, object): [a, b, c, d, e]

#### （d）利用remove_categories移除

In [14]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.remove_categories(['d'])

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (3, object): [a, b, c]

#### （e）删除元素值未出现的分类类型

In [15]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.remove_unused_categories()

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (3, object): [a, b, c]

## 二、分类变量的排序
#### 前面提到，分类数据类型被分为有序和无序，这非常好理解，例如分数区间的高低是有序变量，考试科目的类别一般看做无序变量

### 1. 序的建立

#### （a）一般来说会将一个序列转为有序变量，可以利用as_ordered方法

In [16]:
s = pd.Series(["a", "d", "c", "a"]).astype('category').cat.as_ordered()
s

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

#### 退化为无序变量，只需要使用as_unordered

In [17]:
s.cat.as_unordered()

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a, c, d]

#### （b）利用set_categories方法中的order参数

In [18]:
pd.Series(["a", "d", "c", "a"]).astype('category').cat.set_categories(['a','e','d'],ordered=True)

0      a
1      d
2    NaN
3      a
dtype: category
Categories (3, object): [a < e < d]

#### （c）利用reorder_categories方法
#### 这个方法的特点在于，新设置的分类必须与原分类为同一集合

In [19]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s.cat.reorder_categories(['a','c','d'],ordered=True)

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

In [20]:
#s.cat.reorder_categories(['a','c'],ordered=True) #报错
#s.cat.reorder_categories(['a','c','d','e'],ordered=True) #报错

### 2. 排序

#### 先前在第1章介绍的值排序和索引排序都是适用的

In [21]:
s = pd.Series(np.random.choice(['perfect','good','fair','bad','awful'],50)).astype('category')
s.cat.set_categories(['perfect','good','fair','bad','awful'][::-1],ordered=True).head()

0    perfect
1       good
2       good
3       good
4    perfect
dtype: category
Categories (5, object): [awful < bad < fair < good < perfect]

In [22]:
s.sort_values(ascending=False).head()

0     perfect
34    perfect
46    perfect
16    perfect
44    perfect
dtype: category
Categories (5, object): [awful, bad, fair, good, perfect]

In [23]:
df_sort = pd.DataFrame({'cat':s.values,'value':np.random.randn(50)}).set_index('cat')  
# np.random.randn返回一个或一组具有标准正态分布的样本
df_sort.head()

,value
cat,
perfect,-2.272530
good,-0.054635
good,-1.272776
good,1.860937
perfect,0.382868


In [24]:
df_sort.sort_index().head()

,value
cat,
awful,-0.138325
awful,1.045044
awful,-1.606542
awful,-0.344773
awful,-0.840987


## 三、分类变量的比较操作

### 1. 与标量或等长序列的比较

#### （a）标量比较

In [25]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s == 'a'

0     True
1    False
2    False
3     True
dtype: bool

#### （b）等长序列比较

In [26]:
s == list('abcd')

0     True
1    False
2     True
3    False
dtype: bool

### 2. 与另一分类变量的比较

#### （a）等式判别（包含等号和不等号）
#### 两个分类变量的等式判别需要满足分类完全相同

In [27]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s == s

0    True
1    True
2    True
3    True
dtype: bool

In [28]:
s != s

0    False
1    False
2    False
3    False
dtype: bool

In [29]:
s_new = s.cat.set_categories(['a','d','e'])
#s == s_new #报错

#### （b）不等式判别（包含>=,<=,<,>）
#### 两个分类变量的不等式判别需要满足两个条件：① 分类完全相同 ② 排序完全相同

In [30]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
#s >= s #报错

In [31]:
s = pd.Series(["a", "d", "c", "a"]).astype('category').cat.reorder_categories(['a','c','d'],ordered=True)
s >= s

0    True
1    True
2    True
3    True
dtype: bool

## 四、问题与练习

### 1、问题

#### 【问题一】 如何使用union_categoricals方法？它的作用是什么？

union_categoricals方法是用来合并两个列表，同时保持分类类型。

In [32]:
from pandas.api.types import union_categoricals
a = pd.Categorical(["b", "c"])
b = pd.Categorical(["a", "b"])
union_categoricals([a, b])

[b, c, a, b]
Categories (3, object): [b, c, a]

参考：https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.api.types.union_categoricals.html

#### 【问题二】 利用concat方法将两个序列纵向拼接，它的结果一定是分类变量吗？什么情况下不是？

In [33]:
a = pd.Categorical(["b", "c"])
b = pd.Categorical(["a", "b"])
pd.concat([pd.Series(a),pd.Series(b)])

0    b
1    c
0    a
1    b
dtype: object

In [34]:
a = pd.Categorical([1, 3])
b = pd.Categorical([5, 7])
pd.concat([pd.Series(a),pd.Series(b)])

0    1
1    3
0    5
1    7
dtype: int64

从上面的例子可以看出，通过concat方法得到的分类变量可以是object或者int。

#### 【问题三】 当使用groupby方法或者value_counts方法时，分类变量的统计结果和普通变量有什么区别？

In [35]:
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


In [36]:
df['Address'].value_counts()

street_4    8
street_6    6
street_2    6
street_5    6
street_7    6
street_1    3
Name: Address, dtype: int64

In [37]:
df['Address'].astype('category').value_counts()

street_4    8
street_7    6
street_6    6
street_5    6
street_2    6
street_1    3
Name: Address, dtype: int64

In [38]:
df['Address'] = df['Address'].astype('category')
for i,data in df.groupby('Address'):
    print(i)
    display(data)

street_1


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
11,S_1,C_3,1302,F,street_1,175,57,87.7,A-
23,S_2,C_2,2204,M,street_1,175,74,47.2,B-


street_2


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
13,S_1,C_3,1304,M,street_2,195,70,85.2,A
30,S_2,C_4,2401,F,street_2,192,62,45.3,A
33,S_2,C_4,2404,F,street_2,160,84,67.7,B


street_4


...

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
5,S_1,C_2,1201,M,street_5,188,68,97.0,A-
8,S_1,C_2,1204,F,street_5,162,63,33.8,B
14,S_1,C_3,1305,F,street_5,187,69,61.7,B-
18,S_2,C_1,2104,F,street_5,159,97,72.2,B+
20,S_2,C_2,2201,M,street_5,193,100,39.1,B
26,S_2,C_3,2302,M,street_5,171,88,32.7,A


street_6


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
7,S_1,C_2,1203,M,street_6,160,53,58.8,A+
9,S_1,C_2,1205,F,street_6,167,63,68.4,B-
16,S_2,C_1,2102,F,street_6,161,61,50.6,B+
28,S_2,C_3,2304,F,street_6,164,81,95.5,A-
32,S_2,C_4,2403,F,street_6,158,60,59.7,B+
34,S_2,C_4,2405,F,street_6,193,54,47.6,B


street_7


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
12,S_1,C_3,1303,M,street_7,188,82,49.7,B
15,S_2,C_1,2101,M,street_7,174,84,83.3,C
21,S_2,C_2,2202,F,street_7,194,77,68.5,B+
24,S_2,C_2,2205,F,street_7,183,76,85.4,B
27,S_2,C_3,2303,F,street_7,190,99,65.9,C
31,S_2,C_4,2402,M,street_7,166,82,48.7,B


上例所述，区别不大。

#### 【问题四】 下面的代码说明了Series创建分类变量的什么“缺陷”？如何避免？（提示：使用Series中的copy参数）

In [39]:
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])
s = pd.Series(cat, name="cat")
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [40]:
s.iloc[0:2] = 10
cat

[10, 10, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

问题就是我们对Series的内容进行更改时，Categories里的内容也进行了更改。所以我们通过在建立Series时设置参数Copy=True,这样就会自动拷贝输入的Categories，然后我们再更改，Categories的相应内容就不会变了。如下：

In [41]:
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])
s = pd.Series(cat, name="cat", copy=True)
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [42]:
s.iloc[0:2] = 10
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

### 2、练习

#### 【练习一】 现继续使用第四章中的地震数据集，请解决以下问题：

In [43]:
pd.read_csv('joyful-pandas-master/data/Earthquake.csv').head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


#### （a）现在将深度分为七个等级：[0,5,10,15,20,30,50,np.inf]，请以深度等级Ⅰ,Ⅱ,Ⅲ,Ⅳ,Ⅴ,Ⅵ,Ⅶ为索引并按照由浅到深的顺序进行排序。

In [44]:
df = pd.read_csv('joyful-pandas-master/data/Earthquake.csv')
df.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [45]:
df['深度'] = pd.cut(df['深度'], [0,5,10,15,20,30,50,np.inf], right=False, labels=['Ⅰ','Ⅱ','Ⅲ','Ⅳ','Ⅴ','Ⅵ','Ⅶ'])
df.set_index('深度').sort_index()

,日期,时间,维度,经度,方向,距离,烈度
深度,,,,,,,
Ⅰ,1999.07.05,12:57:31 AM,41.24,32.78,north_west,0.9,0.0
Ⅰ,1994.12.24,12:40:49 AM,37.48,28.35,north_east,1.0,0.0
Ⅰ,2001.12.12,12:59:55 AM,37.42,37.28,north_east,3.2,0.0
Ⅰ,2017.04.21,12:25:56 AM,38.78,29.06,south_east,1.5,3.8
Ⅰ,1992.08.21,12:10:52 AM,37.61,27.48,north_east,1.0,0.0
...,...,...,...,...,...,...,...
Ⅶ,1981.08.04,12:52:19 AM,38.90,37.00,north,3.7,0.0
Ⅶ,1968.03.21,12:42:51 AM,38.80,27.60,south,0.6,4.3
Ⅶ,1929.04.27,12:18:06 AM,40.51,31.43,south_west,1.9,4.8


#### （b）在（a）的基础上，将烈度分为4个等级：[0,3,4,5,np.inf]，依次对南部地区的深度和烈度等级建立多级索引排序。

In [46]:
df['烈度'] = pd.cut(df['烈度'], [0,3,4,5,np.inf], right=False)
df.set_index(['深度','烈度']).sort_index()

日期           时间     维度     经度          方向    距离
深度 烈度                                                                 
Ⅰ  [0.0, 3.0)  1978.05.07  12:41:37 AM  38.58  27.61  south_west   0.1
   [0.0, 3.0)  2000.02.07  12:11:45 AM  40.05  34.07  south_east   0.1
   [0.0, 3.0)  1985.01.28  12:20:56 AM  38.85  29.06  north_east   0.1
   [0.0, 3.0)  1999.11.16  12:49:39 AM  40.70  31.63        west   0.2
   [0.0, 3.0)  2006.06.09  12:53:05 AM  39.67  38.99        west   0.2
...                   ...          ...    ...    ...         ...   ...
Ⅶ  [5.0, inf)  1966.08.19  12:15:14 AM  39.41  41.30  south_east   7.0
   [5.0, inf)  1934.06.19  12:43:16 AM  37.86  31.13  north_east   8.6
   [5.0, inf)  1958.05.09  12:40:57 AM  36.61  27.60  south_east  10.5
   [5.0, inf)  1925.01.09  12:38:34 AM  41.33  43.41        east  20.4
   [5.0, inf)  1949.10.13  12:28:39 AM  36.75  44.47  south_east  31.8

[10062 rows x 6 columns]

#### 【练习二】 对于分类变量而言，调用第4章中的变形函数会出现一个BUG（目前的版本下还未修复）：例如对于crosstab函数，按照[官方文档的说法](https://pandas.pydata.org/pandas-docs/version/1.0.0/user_guide/reshaping.html#cross-tabulations)，即使没有出现的变量也会在变形后的汇总结果中出现，但事实上并不是这样，比如下面的例子就缺少了原本应该出现的行'c'和列'f'。基于这一问题，请尝试设计my_crosstab函数，在功能上能够返回正确的结果。

In [47]:
foo = pd.Categorical(['a', 'b'], categories=['a', 'b', 'c'])
bar = pd.Categorical(['d', 'e'], categories=['d', 'e', 'f'])
pd.crosstab(foo, bar)

col_0,d,e
row_0,,
a,1,0
b,0,1


In [48]:
def my_crosstab(foo,bar):
    num = len(foo)
    s1 = pd.Series([i for i in list(foo.categories.union(set(foo)))])
    s2 = [i for i in list(bar.categories.union(set(bar)))]
    df = pd.DataFrame({i:[0]*len(s1) for i in s2},index=s1)
    for i in range(num):
        df.at[foo[i],bar[i]] += 1
    return df
my_crosstab(foo,bar)

,d,e,f
a,1,0,0
b,0,1,0
c,0,0,0
